In [7]:
import yaml
import os

config_file = "destination-rules.yaml"
kubectl_apply_cmd = "kubectl apply -f "

In [56]:
# https://istio.io/latest/docs/reference/config/networking/destination-rule/#ConnectionPoolSettings
class IstioConfig:
    def __init__(self, file_name):
        self.file_name = file_name
        with open(file_name, "r") as stream:
            docs = yaml.load_all(stream, Loader=yaml.FullLoader)
            self.docs = list(docs)
    
    # save to yaml file
    def save(self):
        with open(self.file_name, "w") as stream:
            yaml.dump_all(self.docs, stream)
    
    # apply to Istio cluster
    def apply(self):
        self.save()
        os.system(kubectl_apply_cmd + self.file_name)
        
    # reset to default config (does not save)
    def reset(self):
        for i in range(self.size()):
            ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["http"] = {}
            ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["tcp"] = {}
    
    def size(self):
        return len(self.docs)
    
    def get_docs(self):
        return self.docs
    
    def _get_http(self, i):
        return ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["http"]
    
    def _get_tcp(self, i):
        return ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["tcp"]
    
    #### tcp ####
    # maxConnections: int32
    # 100
    def set_tcp_max_connections(self, i, val):
        self._get_tcp(i)["maxConnections"] = val
    
    # connectTimeout: int (unit ms)
    # 30
    def set_tcp_connect_timeout(self, i, val):
        self._get_tcp(i)["connectTimeout"] = str(val) + "ms"
        
    # TcpKeepalive time: int (unit ms)
    # 
    def set_tcp_keeplive_probes(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["time"] = val
        else:
            tcp["tcpKeepalive"] = {"time": val}
            
    # TcpKeepalive time: int (unit ms)
    # 7200s
    def set_tcp_keeplive_time(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["time"] = val
        else:
            tcp["tcpKeepalive"] = {"time": val}
            
    # TcpKeepalive interval: int (unit ms)
    # 75s
    def set_tcp_keeplive_interval(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["interval"] = val
        else:
            tcp["tcpKeepalive"] = {"interval": val}
    
    #### http ####
    # http1MaxPendingRequests: int32
    def set_http_http1_max_pending_requests(self, i, val):
        self._get_http(i)["http1MaxPendingRequests"] = val
    
    # http2MaxRequests: int32
    def set_http_http2_max_requests(self, i, val):
        self._get_http(i)["http2MaxRequests"] = val
    
    # maxRequestsPerConnection: int32
    def set_http_max_requests_per_connection(self, i, val):
        self._get_http(i)["maxRequestsPerConnection"] = val
        
    # maxRetries: int32
    def set_http_max_retries(self, i, val):
        self._get_http(i)["maxRetries"] = val
    
    # idleTimeout : int (unit ms)
    def set_http_idle_timeout(self, i, val):
        self._get_http(i)["idleTimeout"] = str(val) + "ms"
        
    # useClientProtocol: bool
    def set_http_use_client_protocol(self, i, val):
        self._get_http(i)["useClientProtocol"] = val

In [57]:
ic = IstioConfig(config_file)

# ic.set_tcp_keeplive_time(0, 150000)
ic.reset()

ic.apply()

destinationrule.networking.istio.io/carts configured
destinationrule.networking.istio.io/carts-db configured
destinationrule.networking.istio.io/catalogue configured
destinationrule.networking.istio.io/catalogue-db configured
destinationrule.networking.istio.io/front-end unchanged
destinationrule.networking.istio.io/orders configured
destinationrule.networking.istio.io/orders-db configured
destinationrule.networking.istio.io/payment configured
destinationrule.networking.istio.io/queue-master configured
destinationrule.networking.istio.io/rabbitmq configured
destinationrule.networking.istio.io/session-db configured
destinationrule.networking.istio.io/shipping configured
destinationrule.networking.istio.io/user configured
destinationrule.networking.istio.io/user-db configured


In [42]:
ic = IstioConfig(config_file)
ic.size()

14

In [48]:
ic = IstioConfig(config_file)
ic.reset()
ic.apply()

destinationrule.networking.istio.io/carts configured
destinationrule.networking.istio.io/carts-db configured
destinationrule.networking.istio.io/catalogue configured
destinationrule.networking.istio.io/catalogue-db configured
destinationrule.networking.istio.io/front-end configured
destinationrule.networking.istio.io/orders configured
destinationrule.networking.istio.io/orders-db configured
destinationrule.networking.istio.io/payment configured
destinationrule.networking.istio.io/queue-master configured
destinationrule.networking.istio.io/rabbitmq configured
destinationrule.networking.istio.io/session-db configured
destinationrule.networking.istio.io/shipping configured
destinationrule.networking.istio.io/user configured
destinationrule.networking.istio.io/user-db configured


In [51]:
import random
temp = []
for i in range(3):
    tem = random.sample(range(1,1000), 6)
    temp.append(tem)
temp

[[683, 267, 4, 611, 508, 332],
 [790, 695, 583, 989, 475, 958],
 [215, 138, 897, 73, 487, 189]]

In [52]:
def generate_data(num_data, num_clients, num_reqs, config_upper_bound):
        '''
            randomly config the istio
            generate load with {num_clients} clients and {num_reqs} requests
            save the performance from Prometheus
            repeat {num_data} times and get {num_data} data
            
            @return states, latencies
        '''
        NUM_ENDPOINTS = 14
        NUM_ACTIONS = 6
        
        states = []
        performances = []
        
        # randomly config the istio
        self.istio_configurator.reset()
        
        for i in range(NUM_ENDPOINTS):
            if i != 4:
                random_vals = random.sample(range(1, upper_bound), 6)
                self.istio_configurator.set_tcp_max_connections(i, random_vals[0])
                self.istio_configurator.set_tcp_connect_timeout(i, random_vals[1])
                self.istio_configurator.set_http_http1_max_pending_requests(i, random_vals[2])
                self.istio_configurator.set_http_http2_max_requests(i, random_vals[3])
                self.istio_configurator.set_http_max_requests_per_connection(i, random_vals[4])
                self.istio_configurator.set_http_max_retries(i, random_vals[5])
                states.append(random_vals)

        self.istio_configurator.apply()
        
        # generate load
        print("num clients {}, num reqs {}".format(num_clients, num_reqs))
        
        output = generate_load(num_clients, num_reqs)
        #print(output)
        time.sleep(3)
        metrics = prometheus_client.custom_query(query="microservices_demo_user_request_latency_microseconds")
        if len(metrics) > 0:
            latencies_99_quantile = list([float(metrics[i]['value'][1]) for i in [2,5,8,14]])
        else:
            # cannot fetch performance metrics
            latencies_99_quantile = [float("nan"), float("nan"), float("nan"), float("nan")]
        performances.append(latencies_99_quantile)
        
        return states, performances